In [2]:
import streamlit as st
import pandas as pd
import base64
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle

st.title('NFL Football Stats and Predictor')

st.markdown("""
This app provides NFL Football stats data!
*  We will also give a prediction on the playoff games for this year!
""")

#st.sidebar.header('Playoff Teams')
#selected_year = st.sidebar.selectbox('Year', list(reversed(range(2015,2021))))
st.sidebar.header('Playoff Teams')

2022-01-10 11:38:45.504 
  command:

    streamlit run /opt/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator(_root_container=1, _provided_cursor=None, _parent=DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None), _block_type=None, _form_data=None)

In [5]:
# Sidebar - Team selection
sorted_unique_team = ["cin","oti","buf","kan","gnb","tam","dal","ram"]
selected_team = st.sidebar.selectbox("Teams", sorted_unique_team)

# Web scraping of NFL player stats
# "https://www.pro-football-reference.com/teams/ram/2021.htm"   
# "https://www.pro-football-reference.com/teams/" + str(team) + "/2021.htm"
#@st.cache
def load_data(team): #year,
    url = "https://www.pro-football-reference.com/teams/" + str(selected_team) + "/2021.htm"
    df = pd.read_html(url, header = 1)
    df = df[1]
    return df
teamstats = load_data(selected_team) #selected_year, 

st.subheader('Display Selected Team Schedule & Game Results')
st.dataframe(teamstats)

# https://discuss.streamlit.io/t/how-to-download-file-in-streamlit/1806
def filedownload(df):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode()).decode()  # strings <-> bytes conversions
    href = f'<a href="data:file/csv;base64,{b64}" download="playerstats.csv">Download CSV File</a>'
    return href

st.markdown(filedownload(teamstats), unsafe_allow_html=True)

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [7]:
col1, col2 = st.columns(2)
col1.header("teams")

col2.header("Prediction")

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [ ]:
button1 = st.button("Run Prediction")
if button1:
    st.write("prediction are being calculated...")
    result = predict_model()
    
    


In [ ]:
#test

In [10]:
bills = st.image("chiefslogo.png", width=40)

In [ ]:
# @st.cache(allow_output_mutation=True)
def load_model(model_name):
    LR = LR.load(model_name)
    return (LR)
LR = load_model('model_name')

In [ ]:
def load_model():
    model = tf.keras.models.load_model("./resourses/model.hdf5")
    return model
model = load_model()

In [6]:
st.sidebar.image("chiefslogo.png", width=40)

DeltaGenerator(_root_container=1, _provided_cursor=None, _parent=DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None), _block_type=None, _form_data=None)

In [11]:
# saving the model to a file to be able to use it later in streamlit. save it in same folder as the .py file
import pickle
pickle.dump(lr, open(' ./model.sav', 'wb'))

In [ ]:
#load model in streamlit
model = pickle.load(open('model.sav', 'rb'))

In [ ]:
# Fit the model on training set
model = LogisticRegression()
model.fit(X_train, Y_train)
# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)